séparer par type d'achat 

les clients achètent de manière générale du pgc et des PRODUITS FRAIS :
favoriser ce type d'achat

supprimer textile et bazar des calculs car il n'y a pas trop de recurrence c'est souvent des produits acheter aléatoirement au moment du besoin

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import dask
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
# Import libraries
from tqdm import tqdm
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

train_dataframes = []
for i in tqdm(range(1, 2)):
    train_dataframes.append(pd.read_csv(f'data-train/train_data_part_{i}.csv'))
train_data = pd.concat(train_dataframes, ignore_index=True)

# free up memory by deleting the dataframes we no longer need
del train_dataframes

# This code reads the data from a CSV file named "products_data.csv" into a pandas DataFrame
products_data = pd.read_csv('data-train/products_data.csv')

# This code reads the data from a CSV file named "test_data.csv" into a pandas DataFrame
test_data = pd.read_csv('data-train/test_data.csv')

100%|██████████| 1/1 [00:20<00:00, 20.58s/it]
C:\Users\jeann\AppData\Local\Temp\ipykernel_13160\3163275204.py:15: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  products_data = pd.read_csv('data-train/products_data.csv')


In [3]:
columns_to_keep = ['date', 'transaction_id', 'customer_id', 'product_id']
train_data = train_data[columns_to_keep]

In [ ]:
import pandas as pd

# Assurer que la colonne 'date' est en format datetime
train_data['date'] = pd.to_datetime(train_data['date'])

# Extraire les informations temporelles
train_data['month'] = train_data['date'].dt.month        # Mois de l'année
train_data['day_of_week'] = train_data['date'].dt.weekday    # jour de la semaine (0 = lundi, 6 = dimanche)
l'information sur les jours de la semaine aurai pu etre interéssante mais pour les données test nous ne savons pas sur quel jour ce passe les achats 

# Afficher un aperçu des données
print(train_data.head())

# Suppression de la colonne date si elle n'est plus nécessaire pour éviter la multicolinéarité
#train_data = train_data.drop(columns=['date'])


        date       transaction_id   customer_id     product_id  month  \
0 2023-11-15  Transaction_1730125  Household_39   Product_5362     11   
1 2022-07-20  Transaction_1560535  Household_39  Product_67174      7   
2 2022-07-20  Transaction_1560535  Household_39  Product_82254      7   
3 2023-11-15  Transaction_1730125  Household_39   Product_3895     11   
4 2022-07-20  Transaction_1560535  Household_39  Product_34014      7   

   day_of_week  hour  
0            2     0  
1            2     0  
2            2     0  
3            2     0  
4            2     0  


In [6]:
# Trier les données par customer_id et date
train_data = train_data.sort_values(by=['customer_id', 'date'])

# Extraire les 10 premières transactions de chaque client
first_50_purchases = train_data.groupby('customer_id').head(50)

# Extraire des variables temporelles des dates
first_50_purchases['date'] = pd.to_datetime(first_50_purchases['date'])
first_50_purchases['month'] = first_50_purchases['date'].dt.month

# Vérifier les données
print(first_50_purchases.head())

# Optionnel : Suppression de la colonne date si non nécessaire
#first_10_purchases = first_10_purchases.drop(columns=['date'])


              date       transaction_id  customer_id     product_id  month  \
1256210 2022-03-01  Transaction_2515147  Household_1  Product_62373      3   
1521128 2022-03-01  Transaction_2515147  Household_1  Product_33145      3   
2048302 2022-03-01  Transaction_2515147  Household_1  Product_68212      3   
3378083 2022-03-01  Transaction_2515147  Household_1  Product_62388      3   
3378089 2022-03-01  Transaction_2515147  Household_1  Product_62136      3   

         day_of_week  hour  
1256210            1     0  
1521128            1     0  
2048302            1     0  
3378083            1     0  
3378089            1     0  


C:\Users\jeann\AppData\Local\Temp\ipykernel_13160\835597002.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_10_purchases['date'] = pd.to_datetime(first_10_purchases['date'])
C:\Users\jeann\AppData\Local\Temp\ipykernel_13160\835597002.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_10_purchases['month'] = first_10_purchases['date'].dt.month


In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical


# Séparer X et y
X = train_data.drop(columns=["product_id"])  # Toutes les colonnes sauf product_id
y = train_data["product_id"]  # La cible (product_id)

# Encodage de la variable cible (product_id)
product_encoder = LabelEncoder()
y_encoded = product_encoder.fit_transform(y)

# Encodage des colonnes catégorielles dans X
categorical_columns = X.select_dtypes(include=["object"]).columns

# Appliquer OneHotEncoder sur les colonnes catégorielles
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore") # sparse was deprecated in 1.2 and replaced with sparse_output
X_encoded = pd.DataFrame(encoder.fit_transform(X[categorical_columns]))

# Ajouter les colonnes numériques
numerical_columns = X.select_dtypes(exclude=["object"]).columns
X_encoded = X_encoded.astype(np.float32)

# Conversion en matrices numpy
X_train = X_encoded.to_numpy()
y_train = to_categorical(y_encoded, num_classes=len(product_encoder.classes_))

print(f"Dimensions de X_train : {X_train.shape}")
print(f"Dimensions de y_train : {y_train.shape}")


In [ ]:
# Encodage des données de test
X_test = test_data[['transaction_id', 'customer_id']]  # Select only the columns present in test_data and used for training

# Get categorical columns present in both train and test data
categorical_columns_test = list(set(categorical_columns) & set(X_test.columns))

# Create missing columns in X_test and fill with a placeholder value (e.g., 0)
# We fill with 0 because the numerical columns are related to the training data and do not exist in the test data.
# Instead of 'missing', we use 0 to represent the absence of these features in the test data.
for col in numerical_columns:
    if col not in X_test.columns:
        X_test[col] = 0

# Create missing columns in X_test and fill with a placeholder value (e.g., 'missing')
for col in categorical_columns:
    if col not in X_test.columns:
        X_test[col] = 'missing'  # Or any other suitable placeholder

# Now you can safely apply the transform
X_test_encoded = pd.DataFrame(encoder.transform(X_test[categorical_columns]))

# Continue with the rest of your code...
x_test_final = pd.concat([X_test_encoded, X_test[numerical_columns].reset_index(drop=True)], axis=1)
X_test_final = x_test_final.to_numpy()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

num_features = X_train.shape[1]  # Nombre de colonnes dans X
num_products = y_train.shape[1]  # Nombre de classes (produits)

model = Sequential()
model.add(Dense(128, input_dim=num_features, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(num_products, activation="softmax"))  # Prédictions des produits

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Entraîner le modèle
model.fit(X_train, y_train, epochs=10, batch_size=32)

In [ ]:
def predict_product(model, top_n=10):
    """
    Génère les produits recommandés pour chaque instance dans X_train.

    Args:
        model: Le modèle entraîné.
        top_n: Le nombre de produits recommandés (par défaut : 10).

    Returns:
        Une liste de produits recommandés pour chaque exemple d'entraînement.
    """
    # Prédire les probabilités pour les données d'entraînement
    prediction = model.predict(X_train)

    # Obtenir les indices des produits les plus probables
    top_products = np.argsort(prediction, axis=1)[:, -top_n:][:, ::-1]

    # Décoder les produits pour retourner les identifiants originaux
    return [product_encoder.inverse_transform(top_products[i]) for i in range(len(top_products))]


In [ ]:
# Générer les recommandations
predicted_products = predict_product(model)

In [ ]:
def hitrate_at_k(test_data, predicted_products, k=10):
    """
    Calcule le Hitrate@K.

    Args:
        test_data: Le jeu de données de test contenant les produits réellement achetés.
        predicted_products: Liste des produits recommandés par le modèle.
        k: Le nombre de recommandations à considérer (par défaut : 10).

    Returns:
        Le score Hitrate@K.
    """
    hits = 0

    for i, true_product in enumerate(test_data["product_id"]):
        if true_product in predicted_products[i][:k]:  # Vérifier si le vrai produit est dans les top K recommandations
            hits += 1

    return hits / len(test_data)


# Calcul du Hitrate@10
hitrate = hitrate_at_k(test_data, predicted_products, k=10)
print(f"Hitrate@10 : {hitrate:.2f}")
